In [ ]:
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

Looking in indexes: https://download.pytorch.org/whl/cu128
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.2/379.2 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.6 MB/s eta 0:00

In [ ]:
import random
import numpy as np
import torch



SEED = 3407
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

max_seq_length = 4096        # Demonstrates long-context + RoPE scaling
dtype = None                # Auto-detect (FP16 on T4, BF16 on A100/L4)
load_in_4bit = True         # Memory-efficient QLoRA-style loading


/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


In [ ]:
assert torch.cuda.is_available(), "Please enable GPU runtime (Colab → Runtime → GPU)"



from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:

"""
We load a pre-quantized 4-bit TinyLlama model provided by Unsloth.
Benefits:
- Very low VRAM usage
- Fast download
- Perfect for Colab / Kaggle / 8–16GB GPUs
"""
BASE_MODEL_NAME = "unsloth/tinyllama-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_NAME,
    max_seq_length=max_seq_length,  # Enables long-context via RoPE scaling
    dtype=dtype,                    # Auto FP16 / BF16
    load_in_4bit=load_in_4bit,      # QLoRA-style memory efficiency
)

==((====))==  Unsloth 2026.1.1: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 4096!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,)

In [ ]:
tokenizer

LlamaTokenizerFast(name_or_path='unsloth/tinyllama-bnb-4bit', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
# =========================
# Apply LoRA Adapters (PEFT)
# =========================
"""
We attach LoRA adapters so that:
- Only ~1–10% parameters are trained
- Base model weights remain frozen
- Training becomes faster and memory efficient

Target modules:
- q_proj, k_proj, v_proj, o_proj → Attention
- gate_proj, up_proj, down_proj → MLP
"""

model = FastLanguageModel.get_peft_model(
    model,
    r=32,                            # LoRA rank (8/16/32/64 are common)
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha=32,                   # Usually = r or 2*r
    lora_dropout=0.0,                # Unsloth recommends 0
    bias="none",                     # Best practice for LoRA
    use_gradient_checkpointing=False,# Set True if model >= 7B
    random_state=3407,               # Reproducibility
)


Unsloth 2026.1.1 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


In [ ]:
model.print_trainable_parameters()

trainable params: 25,231,360 || all params: 1,125,279,744 || trainable%: 2.2422


In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight
base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight
base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight
base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight
base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weight
base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight
base_model.model.model.layers.0.mlp.down_proj.lora_B.default.weight
base_model.model.model.layer

In [ ]:
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())

print(f"Trainable: {trainable}")
print(f"Total: {total}")
print(f"Percent: {100 * trainable / total:.2f}%")

Trainable: 25231360
Total: 640837632
Percent: 3.94%


In [ ]:

next(model.parameters()).device

device(type='cuda', index=0)

In [ ]:
next(model.parameters()).dtype

from peft import PeftModel
print(isinstance(model, PeftModel))
model.peft_config

True


{'default': LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping={'base_model_class': 'LlamaForCausalLM', 'parent_library': 'transformers.models.llama.modeling_llama', 'unsloth_fixed': True}, peft_version='0.18.0', base_model_name_or_path='unsloth/tinyllama-bnb-4bit', revision=None, inference_mode=False, r=32, target_modules={'v_proj', 'q_proj', 'up_proj', 'o_proj', 'down_proj', 'gate_proj', 'k_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, alora_invocation_tokens=None, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False)

In [ ]:
import torch
torch.cuda.memory_summary()


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    826 MiB |    995 MiB |   2071 MiB |   1245 MiB |\n|       from large pool |    703 MiB |    994 MiB |   1945 MiB |   1242 MiB |\n|       from small pool |    123 MiB |    123 MiB |    126 MiB |      3 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |    826 MiB |    995 MiB |   2071 MiB |   1245 MiB |\n|       from large pool |    703 MiB |    994 MiB |

In [ ]:
# =========================
# Dataset Preparation (Alpaca Format)
# =========================
"""
We convert raw Alpaca data into a single 'text' field.
IMPORTANT:
- EOS token is mandatory, otherwise generation may never stop.
"""

EOS_TOKEN = tokenizer.eos_token

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""



def format_data(examples):
    texts = []
    for instruction, input_text, output in zip(
        examples["instruction"],
        examples["input"],
        examples["output"],
    ):
        text = alpaca_prompt.format(
            instruction,
            input_text,
            output
        ) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Load dataset
dataset = load_dataset("yahma/alpaca-cleaned", split="train")

# Shuffle + small subset for fast demo training
dataset = dataset.shuffle(seed=3407).select(range(1500))

# Apply formatting
dataset = dataset.map(
    format_data,
    batched=True,
    remove_columns=dataset.column_names,  # keep only 'text'
)


README.md: 0.00B [00:00, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1500
})

In [ ]:
dataset["text"][0]

'Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\n\n### Instruction:\nList 5 popular dishes in US.\n\n### Input:\n\n\n### Response:\n1. Hamburger: A classic American dish consisting of a beef patty served on a bun, often topped with cheese and various toppings such as lettuce, tomatoes, onions, and condiments like ketchup and mustard.\n\n2. Macaroni & Cheese: A comforting dish made from elbow macaroni mixed with a creamy cheese sauce and often baked until golden brown and bubbly.\n\n3. Fried Chicken: A southern staple, fried chicken is typically made from pieces of chicken that are coated in a seasoned batter and deep-fried until crispy and golden brown.\n\n4. Pizza: Pizza is a beloved food in the United States, with toppings ranging from classic pepperoni and cheese to more unique combinations like pineapple and ham.\n\n5. Apple Pie: A classic American dessert made from a fla

In [ ]:

# psutil = Process & System Utilities
!pip install psutil

import time, psutil

# Clears unused GPU memory
# Frees cached tensors
# Prevents fake VRAM spikes
# Reset GPU stats

# peak GPU memory used
# This line resets peak counter so:
# You measure only THIS training run
# Not previous runs / notebooks

torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()


In [ ]:
# your current Python notebook process
# Used to measure CPU RAM consumption
process = psutil.Process()
train_start_time = time.time()
cpu_ram_before = process.memory_info().rss / 1024**3  # GB

In [ ]:
process

psutil.Process(pid=958, name='python3', status='running', started='08:45:50')

In [ ]:

# =========================
# Supervised Fine-Tuning (SFT)
# =========================
"""
We use TRL's SFTTrainer with Unsloth optimizations.
Key features:
- Sequence packing → better GPU utilization
- 8-bit optimizer → lower VRAM usage
- Gradient accumulation → simulates larger batch size
"""

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    packing=True,  # Packs multiple short samples into one sequence
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        num_train_epochs=1,
        learning_rate=2e-5,
        warmup_ratio=0.1,               # Stabilizes early training
        optim="adamw_8bit",             # Memory efficient optimizer
        logging_steps=10,               # Progress visibility
        seed=3407,                      # Reproducibility
        output_dir="outputs",
        report_to="none",               # Disable WandB by default
    ),
)

trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/1500 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,500 | Num Epochs = 1 | Total steps = 188
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 25,231,360 of 1,125,279,744 (2.24% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.058000
20,1.988400
30,2.207600
40,2.022700
50,2.048200
60,2.121000
70,2.019800
80,2.133200
90,2.027900
100,2.011200


TrainOutput(global_step=188, training_loss=2.0524675440280995, metrics={'train_runtime': 383.9977, 'train_samples_per_second': 3.906, 'train_steps_per_second': 0.49, 'total_flos': 2239711176781824.0, 'train_loss': 2.0524675440280995, 'epoch': 1.0})

In [ ]:
train_end_time = time.time()
cpu_ram_after = process.memory_info().rss / 1024**3  # GB

training_time_sec = round(train_end_time - train_start_time, 2)
peak_gpu_vram_gb = round(torch.cuda.max_memory_reserved() / 1024**3, 3)
cpu_ram_used_gb = round(cpu_ram_after - cpu_ram_before, 3)

print("===== UNSLOTH TRAINING STATS =====")
print(f"Training time (sec): {training_time_sec}")
print(f"Peak GPU VRAM (GB): {peak_gpu_vram_gb}")
print(f"CPU RAM used (GB): {cpu_ram_used_gb}")

===== UNSLOTH TRAINING STATS =====
Training time (sec): 412.57
Peak GPU VRAM (GB): 1.928
CPU RAM used (GB): 0.919


In [ ]:
# =========================
# Inference (Fast Generation)
# =========================
"""
Unsloth provides 2× faster inference using optimized kernels.
IMPORTANT:
- Always call FastLanguageModel.for_inference(model)
- Disable gradients for inference
"""

FastLanguageModel.for_inference(model)

prompt = alpaca_prompt.format(
    "Continue the Fibonacci sequence",
    "1, 1, 2, 3, 5, 8",
    ""
)

inputs = tokenizer(
    prompt,
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=64,
        use_cache=True,     # Faster decoding
        do_sample=False     # Deterministic output for demo
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))




# =========================
# Save LoRA Adapters
# =========================
"""
This saves ONLY the LoRA adapters, not the full base model.
Benefits:
- Very small size (~50–200 MB)
- Can be merged later into FP16 / 4-bit / GGUF
- Easy to share or version-control
"""

LORA_SAVE_PATH = "lora_model"

model.save_pretrained(LORA_SAVE_PATH)
tokenizer.save_pretrained(LORA_SAVE_PATH)

print(f"LoRA adapters saved at: {LORA_SAVE_PATH}")



# Hugging Face baseline (NO Unsloth)

# model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# HF Transformers
# bitsandbytes 4-bit
# PEFT LoRA
# Standard PyTorch kernels


# Unsloth benchmark

# model_name = "unsloth/tinyllama-bnb-4bit"

# Same base TinyLlama
# But:
# Pre-quantized
# Triton kernels
# Manual autograd
# Auto packing
# Faster attention


# When NOT to use Unsloth:
# - If you need heavy multi-node distributed training
# - If you want no-code UI only (LLaMA-Factory better)
# - If training classical ML models (not LLMs)

Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
Continue the Fibonacci sequence

### Input:
1, 1, 2, 3, 5, 8

### Response:
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1
LoRA adapters saved at: lora_model


In [31]:
from unsloth import FastLanguageModel
from peft import PeftModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
)

model = PeftModel.from_pretrained(model, "lora_model")


==((====))==  Unsloth 2026.1.1: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [32]:
model = model.merge_and_unload()
FastLanguageModel.for_inference(model)


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,)

In [36]:
import os
os.makedirs("tinyllama-finetuned", exist_ok=True)


In [37]:
model.save_pretrained_gguf(
    save_directory="tinyllama-finetuned",
    tokenizer=tokenizer,
    quantization_method="q4_k_m"
)


Unsloth: Merging model weights to 16-bit format...
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...


RuntimeError: Unsloth: GGUF conversion failed: Unsloth: `config.json` does not exist inside `tinyllama-finetuned`.

In [38]:
model.save_pretrained("tinyllama-finetuned")
tokenizer.save_pretrained("tinyllama-finetuned")


('tinyllama-finetuned/tokenizer_config.json',
 'tinyllama-finetuned/special_tokens_map.json',
 'tinyllama-finetuned/tokenizer.model',
 'tinyllama-finetuned/added_tokens.json',
 'tinyllama-finetuned/tokenizer.json')

In [39]:
import os
print(os.listdir("tinyllama-finetuned"))


['model.safetensors', 'tokenizer.json', 'tokenizer.model', 'config.json', 'special_tokens_map.json', 'tokenizer_config.json', 'generation_config.json']


In [44]:
ls tinyllama-finetuned


config.json             special_tokens_map.json  tokenizer.model
generation_config.json  tokenizer_config.json
model.safetensors       tokenizer.json


In [46]:
!sudo apt-get update


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [47]:
!sudo apt-get install python3.10 -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.10 is already the newest version (3.10.12-1~22.04.12).
python3.10 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 54 not upgraded.


In [48]:
!python3.10 --version


Python 3.10.12


In [50]:
from transformers import AutoConfig


In [55]:
!sudo apt-get install python3-pip -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  javascript-common libjs-sphinxdoc libjs-underscore python3-dev
  python3-pkg-resources python3-setuptools python3-wheel python3.10-dev
Suggested packages:
  apache2 | lighttpd | httpd python-setuptools-doc
The following NEW packages will be installed:
  javascript-common libjs-sphinxdoc libjs-underscore python3-dev python3-pip
  python3-setuptools python3-wheel python3.10-dev
The following packages will be upgraded:
  python3-pkg-resources
1 upgraded, 8 newly installed, 0 to remove and 53 not upgraded.
Need to get 2,816 kB of archives.
After this operation, 10.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 javascript-common all 11+nmu1 [5,936 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libjs-underscore all 1.13.2~dfsg-2 [118 kB]
Get:3 https://ppa.launchpadcontent.net/ub

In [56]:
!python3.10 -m pip install -U transformers safetensors sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.2/507.2 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.3/770.3 KB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 KB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 791.7/791.7 KB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [59]:
!python3.10 -m pip install torch --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 53.2 MB/s eta 0:00:00


In [60]:
!python3.10 llama.cpp/convert_hf_to_gguf.py \
  tinyllama-finetuned \
  --outfile tinyllama-finetuned-q4_k_m.gguf \
  --outtype q4_k_m


usage: convert_hf_to_gguf.py
       [-h]
       [--vocab-only]
       [--outfile OUTFILE]
       [--outtype {f32,f16,bf16,q8_0,tq1_0,tq2_0,auto}]
       [--bigendian]
       [--use-temp-file]
       [--no-lazy]
       [--model-name MODEL_NAME]
       [--verbose]
       [--split-max-tensors SPLIT_MAX_TENSORS]
       [--split-max-size SPLIT_MAX_SIZE]
       [--dry-run]
       [--no-tensor-first-split]
       [--metadata METADATA]
       [--print-supported-models]
       [--remote]
       [--mmproj]
       [--mistral-format]
       [--disable-mistral-community-chat-template]
       [--sentence-transformers-dense-modules]
       [model]
convert_hf_to_gguf.py: error: argument --outtype: invalid choice: 'q4_k_m' (choose from 'f32', 'f16', 'bf16', 'q8_0', 'tq1_0', 'tq2_0', 'auto')


In [61]:
!python3.10 llama.cpp/convert_hf_to_gguf.py \
  tinyllama-finetuned \
  --outfile tinyllama-finetuned-f16.gguf \
  --outtype f16


INFO:hf-to-gguf:Loading model: tinyllama-finetuned
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
Traceback (most recent call last):
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 11112, in <module>
    main()
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 11089, in main
    model_instance = model_class(dir_model, output_type, fname_out,
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 2514, in __init__
    super().__init__(*args, **kwargs)
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 759, in __init__
    super().__init__(*args, **kwargs)
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 163, in __init__
    self.dequant_model()
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 472, in dequant_model
    raise NotImplementedError(f"Quant method is not yet supported: {quant_method!r}")
NotImplementedError: Quant method is not yet supported: 'bitsandbytes'


In [62]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # change if different
LORA_PATH = "/content/lora_model"
OUTPUT_DIR = "/content/tinyllama-finetuned-fp16"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"   # GPU ok
)

model = PeftModel.from_pretrained(base_model, LORA_PATH)
model = model.merge_and_unload()  # 🔥 THIS IS THE KEY LINE

model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

('/content/tinyllama-finetuned-fp16/tokenizer_config.json',
 '/content/tinyllama-finetuned-fp16/special_tokens_map.json',
 '/content/tinyllama-finetuned-fp16/chat_template.jinja',
 '/content/tinyllama-finetuned-fp16/tokenizer.model',
 '/content/tinyllama-finetuned-fp16/added_tokens.json',
 '/content/tinyllama-finetuned-fp16/tokenizer.json')

In [63]:
!ls /content/tinyllama-finetuned-fp16


chat_template.jinja	model.safetensors	 tokenizer.json
config.json		special_tokens_map.json  tokenizer.model
generation_config.json	tokenizer_config.json


In [64]:
!python3.10 llama.cpp/convert_hf_to_gguf.py \
  /content/tinyllama-finetuned-fp16 \
  --outfile tinyllama-finetuned-f16.gguf \
  --outtype f16


INFO:hf-to-gguf:Loading model: tinyllama-finetuned-fp16
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:output.weight,               torch.float16 --> F16, shape = {2048, 32000}
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {2048, 32000}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> F16, shape = {5632, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> F16, shape = {2048, 256}
INFO:hf-to-gguf:blk.0.at